# Notebook para acesso ao Receita Data - dados de viajantes

In [1]:
import getpass
import pandas as pd
from datalake_serpro import handler_dl

In [2]:
user = getpass.getuser() + '@DATALAKE.SERPRO'
senha = getpass.getpass(f'Senha para kinit {user}:')

Senha para kinit 25052288840@DATALAKE.SERPRO:········


In [3]:
commands = f'echo "{senha}" | /opt/anaconda3/bin/kinit {user}'
! {commands}

Password for 25052288840@DATALAKE.SERPRO: 


In [4]:
! /opt/anaconda3/bin/klist

Ticket cache: FILE:/tmp/krb5cc_1227403730
Default principal: 25052288840@DATALAKE.SERPRO

Valid starting       Expires              Service principal
06/07/2021 17:57:02  06/08/2021 03:57:02  krbtgt/DATALAKE.SERPRO@DATALAKE.SERPRO
	renew until 06/14/2021 17:57:02


# Testes

In [5]:
con = handler_dl(DEBUG=True, timeout=300)

Conectado ao Servidor impalaharfb.srv.datalake.serpro.


In [6]:
cpfs = pd.read_csv('viagem.csv', header=None, names=['cpf'])

In [7]:
cpfs.cpf.values

array(['cpf_cnpj', '24500272895', '31319748000185', '71678031119',
       '14838636000111', '34531284000164', '15149499897', '71666466166',
       '71671615190', '33696488810', '08652438757', '74182593000190',
       '22692307879', '24138981000104', '00963519000175',
       '05342805000137', '84574062720', '37121715015', '51542161568',
       '12211733000154', '37404903904', '00017889650', '29652717000108',
       '01041693000123', '01003363105', '14706880000120',
       '58277831000195', '93165269068', '00033971145', '02464443557',
       '47102497687', '58393528968', '47862580753', '05948334856',
       '04860363671', '05401876408', '74405721149', '60652772153',
       '68722192620', '02886427000164', '86846847000379', '75828839349',
       '34628478805', '07424168878', '53213866920', '00387243089',
       '71676285148', '14661049888', '70570914434', '74016628172',
       '24497587800', '84717874900', '22220546802', '05500075632',
       '82330441991', '23659544000165', '35248581800'

In [23]:
'", "'.join(cpfs.cpf.values[:10])

'71678031119", "29539415845", "15149499897", "71666466166", "28189019520", "00624937950", "71671615190", "65638336834", "33696488810", "08652438757'

In [27]:
df_vu

,numero_vu,cpf_vu
0,2.014093e+16,65638336834
1,2.014082e+16,33696488810
2,2.017022e+16,29539415845
3,2.014112e+16,08652438757
4,2.015111e+16,15149499897
5,2.017081e+16,00624937950
6,2.018040e+16,28189019520


In [33]:
lista_vus = ['%d' % numero_vu for numero_vu in df_vu.numero_vu.values]
lista_vus

['20140925215071420',
 '20140822203576048',
 '20170224001161216',
 '20141122182006344',
 '20151113074532508',
 '20170812145861504',
 '20180402083729272']

In [34]:
sql_viagens = 'SELECT * FROM coana.edbv_dados_voos WHERE apvj_vuvg_cd in (%s)' % ', '.join(lista_vus)
df_viagens = pd.read_sql(sql_viagens, con)

In [35]:
df_viagens

,apvj_vuvg_cd,apvj_apva_apvo_dt_chegada,apvj_cd_loc_reserva,apvj_cd_local_embarque,apvj_cd_local_destino,apvj_cd_agencia_gov,apvj_qt_bagagem,apvj_qt_peso_bag_kg,apco_nm,apvj_apva_apvo_nr_voo
0,2.017022e+16,2019-07-04 08:20:00,GRXMZT,MIA,GRU,None,NaN,None,American Airlines,995
1,2.017022e+16,2020-11-01 05:50:00,XLZHYF,GRU,MIA,None,2.0,None,American Airlines,906
2,2.014093e+16,2014-09-25 21:50:00,2G282U,EZE,GRU,None,NaN,None,TAM Mercosur,721
3,2.014093e+16,2019-11-08 05:20:00,IQQVIJ,BOG,GRU,None,NaN,None,TAM Brazilian Airlines,8001
4,2.014093e+16,2018-09-18 10:45:00,PEOHIK,GRU,HAJ,None,2.0,None,Lufthansa,507
...,...,...,...,...,...,...,...,...,...,...
118,2.017081e+16,2017-08-13 06:25:00,NYKSH,GRU,LIS,None,1.0,None,Iberia Airlines,6824
119,2.017022e+16,2018-06-24 04:55:00,FOOXDH,GRU,MIA,None,1.0,None,American Airlines,930
120,2.017022e+16,2015-11-22 07:30:00,RQELIC,MIA,GRU,None,2.0,None,American Airlines,929
121,2.014093e+16,2016-08-01 10:45:00,ZAAHGZ,GRU,MVD,None,1.0,None,TAM Brazilian Airlines,8044


In [39]:
df_final = df_viagens.merge(df_vu, left_on='apvj_vuvg_cd', right_on='numero_vu')

In [40]:
df_final.head()

,apvj_vuvg_cd,apvj_apva_apvo_dt_chegada,apvj_cd_loc_reserva,apvj_cd_local_embarque,apvj_cd_local_destino,apvj_cd_agencia_gov,apvj_qt_bagagem,apvj_qt_peso_bag_kg,apco_nm,apvj_apva_apvo_nr_voo,numero_vu,cpf_vu
0,2.017022e+16,2019-07-04 08:20:00,GRXMZT,MIA,GRU,None,NaN,None,American Airlines,995,2.017022e+16,29539415845
1,2.017022e+16,2020-11-01 05:50:00,XLZHYF,GRU,MIA,None,2.0,None,American Airlines,906,2.017022e+16,29539415845
2,2.017022e+16,2017-11-25 10:25:00,PXBGRX,MIA,GRU,None,3.0,None,American Airlines,995,2.017022e+16,29539415845
3,2.017022e+16,2017-05-29 06:55:00,CMZPYD,GRU,MIA,None,1.0,None,American Airlines,906,2.017022e+16,29539415845
4,2.017022e+16,2020-03-18 06:40:00,MGTMXF,GRU,MIA,None,1.0,None,American Airlines,906,2.017022e+16,29539415845


In [36]:
con.close()

# Rodar

In [5]:
def get_viagens_cpfs(lista_cpfs):
    sql_vu = 'SELECT DISTINCT cpf_vu as cpf, numero_vu FROM coana.edbv_viajante_unico_geral WHERE cpf_vu in ("%s")'
    sql_viagens = 'SELECT * FROM coana.edbv_dados_voos WHERE apvj_vuvg_cd in (%s)'
    df_vu = pd.read_sql(sql_vu % '", "'.join(lista_cpfs), con)
    lista_vus = ['%d' % numero_vu for numero_vu in df_vu.numero_vu.values]
    df_viagens = pd.read_sql(sql_viagens % ', '.join(lista_vus), con)
    df_final = df_vu.merge(df_viagens, right_on='apvj_vuvg_cd', left_on='numero_vu')
    return df_final

In [6]:
def get_nomes_cnpjs(lista_cpfs):
    sql_cnpjs ='SELECT DISTINCT b_cd_cnpj_emph as cnpj, nm_cnpj_emph_empresarial as nome ' + \
               'FROM cnpj.wd_cnpj_esth where b_cd_cnpj_emph in (%s)'
    lista_cnpjs = []
    for row in lista_cpfs:
        try:
            item = str(int(row[:8]))
            lista_cnpjs.append(item)
        except:
            pass
    df_cnpjs = pd.read_sql(sql_cnpjs % ', '.join(lista_cnpjs), con)
    return df_cnpjs

def get_nomes_cpfs(lista_cpfs):
    sql_cpfs = 'SELECT DISTINCT b_cd_cnpf_cpf as cpf, nm_cnpf_cpfa_nome as nome ' + \
               ' FROM cnpf.wd_cnpf_cpfa where b_cd_cnpf_cpf in (%s)'
    lista = []
    for row in lista_cpfs:
        try:
            item = str(int(row))
            lista.append(item)
        except:
            pass
    df_cpfs = pd.read_sql(sql_cpfs % ', '.join(lista), con)
    return df_cpfs

In [7]:
cpfs = pd.read_csv('viagem.csv', header=None, names=['cpf'])
cpfs2 = pd.read_csv('pessoa.csv', header=None, names=['cpf'])
with handler_dl(DEBUG=True, timeout=300) as con:
    df_viagens = get_viagens_cpfs(cpfs.cpf.values)
    df_cnpjs = get_nomes_cnpjs(cpfs2.cpf.values)
    df_cpfs = get_nomes_cpfs(cpfs2.cpf.values)

df_viagens.to_csv('viagens.csv')
df_cnpjs.to_csv('cnpjs.csv')
df_cpfs.to_csv('cpfs.csv')

Conectado ao Servidor impalaharfb.srv.datalake.serpro.
